# **Preparation Notebook**



---
## Setup Environment

In [29]:
# DO NOT MODIFY THE CODE IN THIS CELL
!pip install -q utstd

from utstd.folders import *
from utstd.ipyrenders import *

at = AtFolder(
    course_code=36106,
    assignment="AT2",
)
at.run()

import warnings
warnings.simplefilter(action='ignore')

ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'C:\\Users\\brohao\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~-~earn\\.libs\\msvcp140.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



You can now save your data files in: c:\Users\brohao\Desktop\UTS\36106\ClassificationModels-36106-AT2\36106\assignment\AT2\data


---
## Student Information

In [30]:
# <Student to fill this section and then remove this comment>
student_name = "Jiayu Hao"
student_id = "25948860"

In [31]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_name', value=student_name)

In [32]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_id', value=student_id)

---
## 0. Python Packages

### 0.a Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [33]:
# <Student to fill this section and then remove this comment>
!pip install scikit-learn
!pip install numpy


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 0.b Import Packages

In [34]:
# DO NOT MODIFY THE CODE IN THIS CELL
import pandas as pd
import altair as alt

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np

---
## A. Feature Selection


## A.0 # Load Data

In [36]:
# DO NOT MODIFY THE CODE IN THIS CELL
try:
  df = pd.read_csv(at.folder_path / "credit_rating.csv")
except Exception as e:
  print(e)

### A.1 Approach 1 — Remove PII & non-causal address blocks

In [37]:
# Choose remove column
pii_cols = [
    "customer_id","prefix","full_name","email","phone_number",
    "secondary_address","building_number","street_name","street_suffix",
    "city","postcode","state_abbr","dob","cc_number","cc_expiry","cc_security_code"
]

In [38]:
# provide an explanation on why you use this approach for feature selection and describe its results
feature_selection_1_insights = """
We removed personally identifiable information (PII) and address-related columns 
because they have no predictive value and may cause privacy or fairness issues. 
These fields, such as names, emails, and credit card numbers, 
are unrelated to credit risk and could introduce bias or data leakage. 

By dropping them, we reduce noise, improve model interpretability, 
and ensure ethical and regulatory compliance. 

This step also lowers dimensionality and keeps only business-relevant, 
reproducible features for modeling.
"""

In [39]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_1_insights', value=feature_selection_1_insights)

### A.2 Approach 2 — Leakage / quasi-labels

In [40]:
# Choose leakage and sensitive column
potential_leak_cols = [
    "cc_interest_rate",     # leakage
    "credit_limit_change",  # leakage
    "credit_mix",           # leakage
    "gender" # sensitive
]

In [41]:
# provide an explanation on why you use this approach for feature selection and describe its results
feature_selection_2_insights = """
We excluded potential leakage and sensitive columns such as cc_interest_rate, credit_limit_change, 
credit_mix, and gender to prevent unfair or unrealistic model performance. 

These variables may contain future or proxy information about the credit rating, 
or raise ethical and compliance risks. 

Removing them helps avoid data leakage, reduce redundancy, and ensure the model remains fair, 
generalizable, and suitable for real business deployment.
"""

In [42]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_2_insights', value=feature_selection_2_insights)

### A.3 Approach3 — Keep core signals + lightweight context

> You can add more cells related to other approaches in this section

In [43]:
# <Student to fill this section and then remove this comment>

In [44]:
base_features = [
    # demographics / context
    "age","birth_country","occupation",
    # ability / behavior
    "annual_income","count_bank_accounts","count_credit_cards",
    "loans_count","avg_days_past_due","count_delayed_payment",
    "count_credit_inquiries","outstanding_debt","credit_ratio",
    "count_credit_history_years","monthly_emi_payment","min_amount_payment",
    "_monthly_invested_amount","payment_behaviour","monthly_balance",
    # low-dimensional loan-type context
    "last_1_loan_type","last_2_loan_type","last_3_loan_type","last_4_loan_type",
    "last_5_loan_type","last_6_loan_type","last_7_loan_type","last_8_loan_type","last_9_loan_type"
]


In [45]:
# provide an explanation on why you use this approach for feature selection and describe its results
feature_selection_3_insights = """
We kept core behavioral and financial features that show strong separation across credit ratings, 
along with limited contextual variables for interpretability. 
These include income, debt, repayment history, and credit inquiries, 
which directly reflect financial ability and discipline. 

Context features like occupation, payment behavior, and recent loan types 
provide useful background without causing leakage. 

This balanced selection keeps key predictive signals while maintaining model 
simplicity and business relevance.
"""

In [46]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_3_insights', value=feature_selection_3_insights)

### A.z Final Selection of Features

In [47]:
target_name = "credit_rating"

In [48]:
# <Student to fill this section and then remove this comment>
drop_cols = set(pii_cols)
features_list = base_features + potential_leak_cols + [target_name]

In [49]:
print("Mainline kept features:", len(features_list))

Mainline kept features: 32


In [50]:
# provide a quick explanation on the features selected
feature_selection_explanations = """
The final feature set retains core financial and behavioral variables 
while removing all PII, address, and sensitive card details to ensure privacy and compliance. 
Key predictors such as income, debt, repayment history, and credit inquiries are kept, 
along with limited context features like occupation and recent loan types. 
The potential_leak_cols are included only to enable controlled comparison experiments 
without additional preprocessing. 
This setup preserves relevant business signals, supports fair evaluation, 
and keeps the data ethically sound and ready for modeling.
"""

In [51]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_explanations', value=feature_selection_explanations)

---
## B. Data Cleaning

In [52]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets
try:
  df_clean = df[features_list].copy()
except Exception as e:
  print(e)

### B.1 Fixing invalid values

In [53]:
# Negative overdue days -> 0
df_clean["avg_days_past_due"] = df_clean["avg_days_past_due"].clip(lower=0)

# Negative EMI -> 0
df_clean["monthly_emi_payment"] = df_clean["monthly_emi_payment"].clip(lower=0)

In [54]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_1_explanations = """
Both avg_days_past_due and monthly_emi_payment should be non-negative by definition, 
as negative values have no business meaning and can mislead the model. 
Setting negatives to zero corrects data quality issues, makes feature distributions more realistic, 
and prevents distorted learning from invalid records. 
This fix improves data integrity and ensures that model behavior aligns with real financial patterns.
"""

In [55]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_1_explanations', value=data_cleaning_1_explanations)

### B.2 Fixing missing values

In [56]:
# Maintain Robustness
cat_cols = [c for c in df_clean.columns if df_clean[c].dtype == "object" and c != target_name]
num_cols = [c for c in df_clean.columns if c != target_name and np.issubdtype(df_clean[c].dtype, np.number)]

for c in cat_cols:
    df_clean[c] = df_clean[c].fillna("Unknown")
for c in num_cols:
    df_clean[c] = df_clean[c].fillna(df_clean[c].median())

In [57]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_2_explanations = """
Even though the current dataset shows no missing values, adding a fallback imputation step 
improves robustness and prevents future errors when new or augmented data include gaps. 

Categorical features use "Unknown" to preserve completeness, 
while numeric features use median values for stability. 

This ensures the pipeline remains reliable, reproducible, and ready for real-world data updates.
"""

In [58]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_2_explanations', value=data_cleaning_2_explanations)

### B.3 Fixing Ratio caps

In [59]:
df_clean["credit_ratio"].describe()

count    12500.000000
mean        32.289001
std          5.119387
min         20.880082
25%         28.010949
50%         32.357246
75%         36.504832
max         48.176599
Name: credit_ratio, dtype: float64

In [60]:
# credit_ratio from percentage to [0,1]
df_clean["credit_ratio"] = (df_clean["credit_ratio"] / 100.0)

In [61]:
df_clean["credit_ratio"].describe()

count    12500.000000
mean         0.322890
std          0.051194
min          0.208801
25%          0.280109
50%          0.323572
75%          0.365048
max          0.481766
Name: credit_ratio, dtype: float64

In [62]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_3_explanations = """
The credit_ratio was converted from percentage to a 0–1 scale for consistency 
with other ratio features like dti and emi_to_income. 

Normalizing this value reduces the effect of large outliers and stabilizes model training, 
especially for linear and distance-based algorithms. 
This adjustment improves numerical consistency and helps the model learn balanced feature relationships.
"""

In [63]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_3_explanations', value=data_cleaning_3_explanations)

### B.4 Fixing Unit

> You can add more cells related to other issues in this section

In [64]:
# <Student to fill this section and then remove this comment>
df_clean["count_credit_history_years"].describe()

count    12500.000000
mean       221.158240
std         99.702928
min          2.000000
25%        144.750000
50%        219.000000
75%        302.000000
max        404.000000
Name: count_credit_history_years, dtype: float64

In [65]:
# 信用历史：把“count_credit_history_years（实为月）”校正成年，并删除原列
df_clean["credit_history_years"] = (df_clean["count_credit_history_years"] / 12.0).clip(lower=0)
df_clean.drop(columns=["count_credit_history_years"], inplace=True)

In [66]:
df_clean["credit_history_years"].describe()

count    12500.000000
mean        18.429853
std          8.308577
min          0.166667
25%         12.062500
50%         18.250000
75%         25.166667
max         33.666667
Name: credit_history_years, dtype: float64

In [67]:
# Provide some explanations on why you believe it is important to fix this issue and its impacts
data_cleaning_4_explanations = """
The original count_credit_history_years values were unrealistically high, 
suggesting the data was recorded in months. 
Converting it to years by dividing by 12 corrects the unit error and aligns it with business meaning. 
The original column was removed to avoid redundancy and multicollinearity. 
This fix improves data accuracy and ensures consistent interpretation of credit history length 
across models.
"""

In [68]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_4_explanations', value=data_cleaning_4_explanations)

---
## C. Feature Engineering

In [69]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets

try:
  df_eng = df_clean.copy()
except Exception as e:
  print(e)

### C.1 New Feature "dti"



In [70]:
eps = 1e-6

In [71]:
df_eng["dti"] = (df_eng["outstanding_debt"].clip(lower=0) * 12.0) / (df_eng["annual_income"].abs() + eps)

In [72]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_1_explanations = """
The debt-to-income (DTI) ratio, calculated as (outstanding_debt * 12) / annual_income, 
measures a customer’s repayment burden relative to income. 
It captures financial stress and is often effective in distinguishing between 
Good, Standard, and Poor credit ratings. 
Adding this feature improves model interpretability and can replace raw debt values 
to reduce collinearity while keeping strong predictive power.
"""

In [73]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_1_explanations', value=feature_engineering_1_explanations)

### C.2 New Feature "emi_to_income"



In [74]:
df_eng["emi_to_income"] = (df_eng["monthly_emi_payment"] * 12.0) / (df_eng["annual_income"].abs() + eps)

In [75]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_2_explanations = """
The emi_to_income ratio, calculated as (monthly_emi_payment * 12) / annual_income, 
shows how much of a customer’s income is used for loan repayments. 
A higher value indicates tighter cash flow and higher default risk. 
This feature complements DTI by providing another view of repayment burden, 
helping the model better capture financial stress and predict credit rating changes.
"""

In [76]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_2_explanations', value=feature_engineering_2_explanations)

### C.3 New Feature Log1p for skewed amounts



In [77]:
for c in ["annual_income","outstanding_debt","monthly_emi_payment","_monthly_invested_amount","monthly_balance"]:
    df_eng[f"log1p__{c}"] = np.log1p(df_eng[c])

In [78]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_3_explanations = """
Long-tailed monetary variables such as income, debt, and balance were log-transformed 
using log1p to reduce skewness and stabilize variance. 
This transformation makes feature distributions more regular and improves linear separability. 
It helps models learn smoother relationships, reduces the impact of extreme values, 
and enhances overall training stability.
"""

In [79]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.4 New Feature Loan-type low-dimensional signals



In [80]:
# find all last_x_loan_type columns(last_ start and _loan_type tail）
loan_cols = [c for c in df.columns if c.startswith("last_") and c.endswith("_loan_type")]
print("Loan type columns:", loan_cols)

# merge those last_x_loan_type columns
loan_types_all = pd.Series(df[loan_cols].values.ravel())

# fix missing values(Robustness)
loan_types_all = loan_types_all.dropna()

# counts number of loan type
loan_type_counts = loan_types_all.value_counts()

# print
print("Unique loan types:", loan_type_counts.shape[0])
print("\nLoan type counts:")
print(loan_type_counts)

Loan type columns: ['last_9_loan_type', 'last_8_loan_type', 'last_7_loan_type', 'last_6_loan_type', 'last_5_loan_type', 'last_4_loan_type', 'last_3_loan_type', 'last_2_loan_type', 'last_1_loan_type']
Unique loan types: 10

Loan type counts:
No Loan                    68339
Payday Loan                 5071
Credit-Builder Loan         5055
Not Specified               4952
Home Equity Loan            4888
Student Loan                4871
Mortgage Loan               4867
Personal Loan               4861
Debt Consolidation Loan     4847
Auto Loan                   4749
Name: count, dtype: int64


In [81]:
# Payday loan
df_eng["flag_payday_recent"] = (
    df_eng[loan_cols].apply(lambda row: int(any(str(v) == "Payday Loan" for v in row)), axis=1)
)

# Not all 'No Loan'
df_eng["flag_has_any_loan"] = (
    df_eng[loan_cols].apply(lambda row: int(any(str(v) != "No Loan" for v in row)), axis=1)
)

# Diversity of loan types
def _diversity(xs):
    s = set([str(v) for v in xs if str(v) != "No Loan"])
    return len(s)
df_eng["loan_type_diversity"] = df_eng[loan_cols].apply(_diversity, axis=1)

In [82]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_4_explanations = """
Instead of one-hot encoding nine loan-type columns, we created three low-dimensional signals: whether the customer has any loan, whether a Payday Loan appears, and the diversity of loan types. These features capture key behavioral information while avoiding feature explosion and overfitting. This approach keeps the data compact, improves interpretability, and preserves essential credit risk signals for modeling.
"""

In [83]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_4_explanations', value=feature_engineering_4_explanations)

### C.5 Fixing Multicollinearity (Remove Original Features)

> You can add more cells related to new features in this section

In [84]:
# Necessary drops: avoid multicollinearity and feature explosion
drop_cols = [
    "annual_income",             # replaced by log1p__annual_income, dti, and emi_to_income
    "outstanding_debt",          # captured in dti
    "monthly_emi_payment",       # captured in emi_to_income
    "_monthly_invested_amount",  # replaced by log1p__ version
    "monthly_balance",           # replaced by log1p__ version
] + loan_cols  # all original last_k_loan_type columns replaced by low-dimensional signals

df_eng.drop(columns=[c for c in drop_cols if c in df_eng.columns], inplace=True)
print("Dropped redundant raw features:", drop_cols)
print(df_eng.columns)

Dropped redundant raw features: ['annual_income', 'outstanding_debt', 'monthly_emi_payment', '_monthly_invested_amount', 'monthly_balance', 'last_9_loan_type', 'last_8_loan_type', 'last_7_loan_type', 'last_6_loan_type', 'last_5_loan_type', 'last_4_loan_type', 'last_3_loan_type', 'last_2_loan_type', 'last_1_loan_type']
Index(['age', 'birth_country', 'occupation', 'count_bank_accounts',
       'count_credit_cards', 'loans_count', 'avg_days_past_due',
       'count_delayed_payment', 'count_credit_inquiries', 'credit_ratio',
       'min_amount_payment', 'payment_behaviour', 'cc_interest_rate',
       'credit_limit_change', 'credit_mix', 'gender', 'credit_rating',
       'credit_history_years', 'dti', 'emi_to_income', 'log1p__annual_income',
       'log1p__outstanding_debt', 'log1p__monthly_emi_payment',
       'log1p___monthly_invested_amount', 'log1p__monthly_balance',
       'flag_payday_recent', 'flag_has_any_loan', 'loan_type_diversity'],
      dtype='object')


In [85]:
# Provide some explanations on why you believe it is important to create this feature and its impacts
feature_engineering_5_explanations = """
Removing redundant raw variables helps prevent multicollinearity and unnecessary model complexity. The dropped features were already represented by transformed or derived variables such as dti, emi_to_income, and log1p__ versions, ensuring no information loss. Deleting the original loan-type columns also avoids feature explosion from one-hot encoding. This step keeps the dataset compact, stable, and more interpretable for modeling.
"""

In [86]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_5_explanations', value=feature_engineering_5_explanations)

---
## D. Data Preparation for Modeling

### D.1 Split Datasets


In [87]:

X = df_eng.drop(columns=[target_name])
y = df_eng[target_name].copy()

X_train_raw, X_temp_raw, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y  # 70% train
)
X_val_raw, X_test_raw, y_val, y_test = train_test_split(
    X_temp_raw, y_temp, test_size=0.50, random_state=42, stratify=y_temp  # 15%/15%
)

print("Shapes:",
      X_train_raw.shape, X_val_raw.shape, X_test_raw.shape,
      y_train.shape, y_val.shape, y_test.shape)

Shapes: (8750, 27) (1875, 27) (1875, 27) (8750,) (1875,) (1875,)


In [88]:
# Provide some explanations on what is the best strategy to use for data splitting for this dataset
data_splitting_explanations = """
A stratified 70/15/15 split was used to keep the class distribution stable across train, validation, and test sets. This approach provides a fair balance between training volume and reliable evaluation. The validation set offers a consistent baseline for simple models, while K-Fold cross-validation can be applied within the training set for more complex tuning. Splitting before encoding or scaling prevents data leakage, ensuring that all transformations are fit only on the training data. This strategy supports both comparability and robustness in model evaluation.
"""

In [89]:
# Do not modify this code
print_tile(size="h3", key='data_splitting_explanations', value=data_splitting_explanations)

### D.2 Data Transformation — OHE + Standardize (Fit by train)


In [90]:
# Categorical → One-Hot Encoding (ignore unknown categories)
categorical_cols = [c for c in X_train_raw.columns if X_train_raw[c].dtype == "object"]
numeric_cols = [c for c in X_train_raw.columns if c not in categorical_cols]

# Define encoders and scalers
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
scaler = StandardScaler(with_mean=True, with_std=True)

# Combine categorical and numerical preprocessing
preprocess = ColumnTransformer(
    transformers=[
        ("cat", ohe, categorical_cols),
        ("num", scaler, numeric_cols),
    ],
    remainder="drop"
)
# Fit on train only to avoid data leakage
X_train = preprocess.fit_transform(X_train_raw)
X_val   = preprocess.transform(X_val_raw)
X_test  = preprocess.transform(X_test_raw)

# Restore column names
ohe_names = list(preprocess.named_transformers_["cat"].get_feature_names_out(categorical_cols)) if categorical_cols else []
all_names = ohe_names + numeric_cols

X_train = pd.DataFrame(X_train, columns=all_names, index=X_train_raw.index)
X_val   = pd.DataFrame(X_val,   columns=all_names, index=X_val_raw.index)
X_test  = pd.DataFrame(X_test,  columns=all_names, index=X_test_raw.index)

y_train = y_train.reset_index(drop=True)
y_val   = y_val.reset_index(drop=True)
y_test  = y_test.reset_index(drop=True)

X_train.head(3)

,birth_country_AU,birth_country_BR,birth_country_CA,birth_country_CO,birth_country_FR,birth_country_IE,birth_country_IN,birth_country_IT,birth_country_NZ,birth_country_PH,...,dti,emi_to_income,log1p__annual_income,log1p__outstanding_debt,log1p__monthly_emi_payment,log1p___monthly_invested_amount,log1p__monthly_balance,flag_payday_recent,flag_has_any_loan,loan_type_diversity
2987,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.596776,-0.973657,-0.206441,-1.104840,-0.559860,0.236443,-0.026572,-0.691585,0.359008,-1.031835
9461,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.185613,-0.027072,-0.441362,0.212834,0.200585,-0.599857,0.051902,-0.691585,0.359008,0.667498
2909,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.278885,-0.693435,-1.031888,0.377586,-0.527591,-1.599893,0.073845,-0.691585,0.359008,-0.465391


In [91]:
# Provide some explanations on why you believe it is important to perform this data transformation and its impacts
data_transformation_1_explanations = """
Categorical variables were one-hot encoded to make them usable by both tree-based and linear models, while numerical features were standardized to improve model stability and comparability. The transformation was fitted only on the training set to prevent information leakage. This approach enhances performance for algorithms sensitive to feature scale, such as Logistic Regression and SVM. Keeping derived features like dti and emi_to_income while removing raw outstanding_debt reduces multicollinearity but preserves financial signal strength.
"""

In [92]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_1_explanations', value=data_transformation_1_explanations)

---
## E. Save Datasets

> Do not change this code

In [93]:
# DO NOT MODIFY THE CODE IN THIS CELL

try:
  X_train.to_csv(at.folder_path / 'X_train.csv', index=False)
  y_train.to_csv(at.folder_path / 'y_train.csv', index=False)

  X_val.to_csv(at.folder_path / 'X_val.csv', index=False)
  y_val.to_csv(at.folder_path / 'y_val.csv', index=False)

  X_test.to_csv(at.folder_path / 'X_test.csv', index=False)
  y_test.to_csv(at.folder_path / 'y_test.csv', index=False)
except Exception as e:
  print(e)